#**Import Statements + Creating Environment**

In [2]:
import numpy as np
import numpy.random as rand

In [3]:
class Cell():
    target = 0
    cell_type = ""
    prob_not_found = 0
    def __init__(self, cell_type, prob_not_found):
        self.cell_type = cell_type
        self.prob_not_found = prob_not_found
        

In [37]:
#create environment
#take input from user
while True:
  try:
    dimension = int(input("Enter dimension of board: "))
    break
  except ValueError:
    print("Must be an integer")
    
env = [[] for i in range(dimension)] # create a list of empty lists
cell_type = ""
prob_not_found = 0
for i in range(dimension):
    for j in range(dimension):
        p = rand.random()
        if p < 0.25:
            cell_type = "flat"
            prob_not_found = 0.1
        elif p <0.50:
            cell_type = "hilly"
            prob_not_found = 0.3
        elif p <0.75:
            cell_type = "forested"
            prob_not_found = 0.7
        else:
            cell_type = "maze of caves"
            prob_not_found = 0.9
        env[i].append(Cell(cell_type, prob_not_found))

Enter dimension of board: 2


In [38]:
#set target in environment
env[1][0].target = 1

In [39]:
print(env[1][0].target)

1


# **Helper Functions**

In [19]:
def search_cell(current_position):
    x = current_position[0]
    y = current_position[1]
    
    if env[x][y].target == 1:
        p = rand.random()
        if p > env[x][y].prob_not_found:
            return True
    return False

In [8]:
#variable assignment tree
def bayes_theorem(current_position, updating_position):
    x = current_position[0]
    y = current_position[1]
    i = updating_position[0]
    j = updating_position[1]
    
    numerator = belief[i][j]
    #print(f"numerator: {numerator}")
    denominator = (1 - belief[x][y]) + belief[x][y]*env[x][y].prob_not_found
    #print(f"denominator: {denominator}")
    return numerator/denominator 

In [9]:
def bayes_current(current_position):
    x = current_position[0]
    y = current_position[1]
    
    numerator = belief[x][y] * env[x][y].prob_not_found
    denominator = (1 - belief[x][y]) + belief[x][y]*env[x][y].prob_not_found
    return numerator/denominator 

In [10]:
def get_valid_neighbors(current_node):
  get_neighbors = [] #list of possible neighbors
  for i in range(-1,2):
    for j in range(-1,2):
      if i==0 and j==0:
        continue
      get_neighbors.append((i,j))
  x, y = current_node
  neighbors = set([])
  for i in get_neighbors:
    neighbor_x = x + i[0]
    neighbor_y = y + i[1]

    if neighbor_x >= dimension or neighbor_x < 0:
      continue
    elif neighbor_y >= dimension or neighbor_y < 0:
      continue
    neighbors.add((neighbor_x, neighbor_y))
  return neighbors

# **Basic Agent 1**

In [92]:
def basic_1(dimension, env):
  belief = np.zeros((dimension,dimension))
  for i in range(dimension):
      for j in range(dimension):
          belief[i][j] = 1/(dimension*dimension)
  print(belief)
  searching = True
  #current_position = [np.random.randint(0,dimension),np.random.randint(0,dimension)]
  current_position = [0,0]
  count = 1
  while searching:
      print(f"current_position: {current_position}")
      #search cell
      query = search_cell(current_position)
      if query:
          print(f"Found target at {current_position}")
          searching = False
          break
      count += 1    
      #update belief
      for i in range(dimension):
          for j in range(dimension):
              if i == current_position[0] and j == current_position[1]:
                  continue
              #Bayes Theorem
              calculated_belief = bayes_theorem(current_position, (i,j))
              belief[i][j] = calculated_belief #update belief
      belief[current_position[0]][current_position[1]] = bayes_current(current_position)
      print(belief)
      
      #choose next position
      max_belief = 0
      total_belief = belief[current_position[0]][current_position[1]]
      for i in get_valid_neighbors(current_position):
          total_belief += belief[i[0]][i[1]]
          if belief[i[0]][i[1]] > max_belief:
              max_belief = belief[i[0]][i[1]]
              current_position = i

      print(f"sum of belief: {total_belief}")
      print(f"sum of belief: {sum(sum(belief))}")
      print()
  return count

In [58]:
path = basic_1(dimension, env)
print(path)

Found target at (1, 0)
2


# **Basic Agent 2**

In [93]:
def basic_2(dimension, env):
  belief = np.zeros((dimension,dimension))
  for i in range(dimension):
      for j in range(dimension):
          belief[i][j] = 1/(dimension*dimension)
  #print(belief)
  searching = True
  #current_position = [np.random.randint(0,dimension),np.random.randint(0,dimension)]
  current_position = [0,0]
  count = 1
  while searching:
      #print(f"current_position: {current_position}")
      #search cell
      query = search_cell(current_position)
      if query:
          print(f"Found target at {current_position}")
          searching = False
          break
      count += 1   
      #update belief
      for i in range(dimension):
          for j in range(dimension):
              if i == current_position[0] and j == current_position[1]:
                  continue
              #Bayes Theorem
              calculated_belief = bayes_theorem(current_position, (i,j))
              belief[i][j] = calculated_belief #update belief
      belief[current_position[0]][current_position[1]] = bayes_current(current_position)
      #print(belief)
      
      #choose next position
      max_belief = 0
      total_belief = belief[current_position[0]][current_position[1]]*(1-env[current_position[0]][current_position[1]].prob_not_found)
      for i in get_valid_neighbors(current_position):
          total_belief += belief[i[0]][i[1]]*(1 - env[i[0]][i[1]].prob_not_found)
          if belief[i[0]][i[1]]*(1-env[i[0]][i[1]].prob_not_found) > max_belief:
              max_belief = belief[i[0]][i[1]]
              current_position = i
      #print(f"sum of belief: {total_belief}")
      #print(f"sum of belief: {sum(sum(belief))}")
      #print()
  return count

In [64]:
path = basic_2(dimension,env)
print(path)

Found target at (1, 0)
3


# **Improved Agent**

In [65]:
def improved_agent(dimension, env):
  belief = np.zeros((dimension,dimension))
  belief_find = np.zeros((dimension,dimension))
  for i in range(dimension):
      for j in range(dimension):
          belief[i][j] = 1/(dimension*dimension)
  for i in range(dimension):
      for j in range(dimension):
          belief_find[i][j] = 1/(dimension*dimension) * (1-env[i][j].prob_not_found)
  a= sum(sum(belief_find))
  for i in range(dimension):
      for j in range(dimension):
          belief_find[i][j] = belief_find[i][j]/a
  #print(belief)
  #print(belief_find)
  searching = True
  current_position = [np.random.randint(0,dimension),np.random.randint(0,dimension)]
  count = 1 
  while searching:
      #print(f"current_position: {current_position}")
      #search cell
      query = search_cell(current_position)
      if query:
          print(f"Found target at {current_position}")
          searching = False
          break
      count += 1
      #update belief
      for i in range(dimension):
          for j in range(dimension):
              if i == current_position[0] and j == current_position[1]:
                  continue
              #Bayes Theorem
              calculated_belief = bayes_theorem(current_position, (i,j))
              belief[i][j] = calculated_belief #update belief
              belief_find[i][j] = belief[i][j]*(1-env[i][j].prob_not_found) #update belief_find

      belief[current_position[0]][current_position[1]] = bayes_current(current_position)
      belief_find[current_position[0]][current_position[1]] = belief[current_position[0]][current_position[1]]*(1-env[current_position[0]][current_position[1]].prob_not_found)
      for i in range(dimension):
          for j in range(dimension):
            belief_find[i][j] = belief_find[i][j]/sum(sum(belief_find))
      #print(belief)
      
      #choose next position
      max_belief = 0
      total_belief = belief[current_position[0]][current_position[1]] + belief_find[current_position[0]][current_position[1]]
      for i in get_valid_neighbors(current_position):
          total_belief += (belief[i[0]][i[1]] + belief_find[i[0]][i[1]])
          if (belief[i[0]][i[1]] + belief_find[i[0]][i[1]]) > max_belief:
              max_belief = (belief[i[0]][i[1]] + belief_find[i[0]][i[1]])
              current_position = i
      #print(f"sum of belief: {total_belief}")
      #print(f"sum of belief: {sum(sum(belief))}")
      #print(f"sum of belief: {sum(sum(belief_find))}")
      #print()

  return count

In [68]:
path = improved_agent(dimension, env)
print(path)

Found target at [1, 0]
1


# **Agent Plots**

**Plot for Basic Agent 1**

In [88]:
num_searches = []

for i in range(10):
  dimension = 2
  env = [[] for i in range(dimension)] # create a list of empty lists
  cell_type = ""
  prob_not_found = 0
  for i in range(dimension):
      for j in range(dimension):
          p = rand.random()
          if p < 0.25:
              cell_type = "flat"
              prob_not_found = 0.1
          elif p <0.50:
              cell_type = "hilly"
              prob_not_found = 0.3
          elif p <0.75:
              cell_type = "forested"
              prob_not_found = 0.7
          else:
              cell_type = "maze of caves"
              prob_not_found = 0.9
          env[i].append(Cell(cell_type, prob_not_found))
  x = np.random.randint(0,dimension)
  y = np.random.randint(0,dimension)
  env[x][y].target = 1
  print((x,y))
  num_searches.append(basic_1(dimension, env))

average_basic_agent_1 = sum(num_searches)/len(num_searches) 
print()
print(average_basic_agent_1) 

Streaming output truncated to the last 5000 lines.
[[0.12 0.36]
 [0.12 0.4 ]]
sum of belief: 1.0
sum of belief: 1.0

current_position: (1, 1)
[[0.13043478 0.39130435]
 [0.43478261 0.04347826]]
sum of belief: 1.0
sum of belief: 1.0

current_position: (1, 0)
[[0.12 0.36]
 [0.12 0.4 ]]
sum of belief: 1.0
sum of belief: 1.0

current_position: (1, 1)
[[0.13043478 0.39130435]
 [0.43478261 0.04347826]]
sum of belief: 1.0
sum of belief: 1.0

current_position: (1, 0)
[[0.12 0.36]
 [0.12 0.4 ]]
sum of belief: 1.0
sum of belief: 1.0

current_position: (1, 1)
[[0.13043478 0.39130435]
 [0.43478261 0.04347826]]
sum of belief: 1.0
sum of belief: 1.0

current_position: (1, 0)
[[0.12 0.36]
 [0.12 0.4 ]]
sum of belief: 1.0
sum of belief: 1.0

current_position: (1, 1)
[[0.13043478 0.39130435]
 [0.43478261 0.04347826]]
sum of belief: 1.0
sum of belief: 1.0

current_position: (1, 0)
[[0.12 0.36]
 [0.12 0.4 ]]
sum of belief: 1.0
sum of belief: 1.0

current_position: (1, 1)
[[0.13043478 0.39130435]
 [0.43478

KeyboardInterrupt: ignored

**Plot for Basic Agent 2**

In [100]:
num_searches = []

for i in range(10):
  dimension = 3
  env = [[] for i in range(dimension)] # create a list of empty lists
  cell_type = ""
  prob_not_found = 0
  for i in range(dimension):
      for j in range(dimension):
          p = rand.random()
          if p < 0.25:
              cell_type = "flat"
              prob_not_found = 0.1
          elif p <0.50:
              cell_type = "hilly"
              prob_not_found = 0.3
          elif p <0.75:
              cell_type = "forested"
              prob_not_found = 0.7
          else:
              cell_type = "maze of caves"
              prob_not_found = 0.9
          env[i].append(Cell(cell_type, prob_not_found))
  x = np.random.randint(0,dimension)
  y = np.random.randint(0,dimension)
  env[x][y].target = 1
  print((x,y))
  num_searches.append(basic_2(dimension, env))

average_basic_agent_2 = sum(num_searches)/len(num_searches) 
print()
print(average_basic_agent_2) 

(2, 2)


IndexError: ignored

**Plot for Improved Agent**

In [ ]:
num_searches = []

for i in range(10):
  dimension = 2
  env = [[] for i in range(dimension)] # create a list of empty lists
  cell_type = ""
  prob_not_found = 0
  for i in range(dimension):
      for j in range(dimension):
          p = rand.random()
          if p < 0.25:
              cell_type = "flat"
              prob_not_found = 0.1
          elif p <0.50:
              cell_type = "hilly"
              prob_not_found = 0.3
          elif p <0.75:
              cell_type = "forested"
              prob_not_found = 0.7
          else:
              cell_type = "maze of caves"
              prob_not_found = 0.9
          env[i].append(Cell(cell_type, prob_not_found))
  x = np.random.randint(0,dimension)
  y = np.random.randint(0,dimension)
  env[x][y].target = 1
  print((x,y))
  num_searches.append(improved_agent(dimension, env))

average_improved_agent = sum(num_searches)/len(num_searches) 
print()
print(average_improved_agent) 